In [190]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

<div style="text-align:center"><span style="color:green">Эпиграф: "Все банки похожи друг на друга, но некоторые не похожи даже на самих себя" © Браниил Дхармс</span></div>

## <div style="text-align:center"><span style="color:red">Импорт данных и их преобразование.</span></div>

*RowNumber — номер строки таблицы  
CustomerId — идентификатор клиента  
Surname — фамилия клиента  
CreditScore — кредитный рейтинг клиента  
Geography — страна клиента  
Gender — пол клиента  
Age — возраст клиента  
Tenure — сколько лет клиент пользуется услугами банка  
Balance — баланс на счетах клиента в банке  
NumOfProducts — количество услуг банка, которые приобрёл клиент  
HasCrCard — есть ли у клиента кредитная карта  
IsActiveMember — есть ли у клиента статус активного клиента банка  
EstimatedSalary — предполагаемая заработная плата клиента  
Exited — статус лояльности*

In [191]:
churn_data = pd.read_csv('data/churn.csv')
churn_df = churn_data.copy()
churn_df.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1, inplace=True)
# churn_df.index.rename('Customers', inplace=True)
display(churn_df.info(), churn_df.sample(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


None

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5879,683,France,Male,41,6,95696.52,2,1,1,184366.14,0
8685,772,France,Male,43,6,0.00,2,1,1,57675.88,0
1076,702,Spain,Male,39,8,0.00,2,1,0,99654.13,0


<span style="color:red">0). Тепловая карта корреляции потенциально значимых признаков</span>

In [192]:
churn_corrs = churn_df.corr().round(2)
mask = np.triu(np.ones_like(churn_corrs, dtype=bool))
churn_corrs.mask(mask, inplace=True)
churn_corrs.drop('CreditScore', axis=0, inplace=True)
churn_corrs.drop('Exited', axis=1, inplace=True)

fig = px.imshow(
    churn_corrs,
    text_auto=True,
    color_continuous_scale='ylorrd',
    aspect="auto",
    title='Тепловая карта корреляции потенциально значимых признаков',
    width=1200,
    height=500,
)
fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()
# fig.write_html("plotly/fig_00.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_00.png?raw=true alt='fig_00.png' style='width:1200px'>

- <span style="color:blue">Пара признаков Возраст-Отток проявляет максимум положительной корреляции (0.29) со значительным отрывом от пар Баланс-Отток (0.12) и Возраст-Активность (0.09)</span>
- <span style="color:blue">По отрицательной взаимосвязи выделяется пара Баланс-Количество Услуг (-0.30) с двойным отрывом от пары Активность-Отток (-0.16)</span>
- <span style="color:blue">С ростом значений Возраста и Баланса возрастает и значение признака Оттока, чем выше Активность, тем значение Оттока ниже</span>
- <span style="color:blue">Кроме того, с ростом значения Баланса снижается значение Количества Приобретённых Услуг</span>

In [193]:
def unique_counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    The function returns a separate DataFrame,
    which shows the count of unique values and
    their type for each column of incoming DataFrame
    Args:
        df (pd.DataFrame)
    Returns:
        pd.DataFrame
    """
    col_series = pd.Series(df.columns)
    ucounts_df = pd.concat(
        [
            col_series,
            col_series.apply(lambda x: df[x].nunique()),
            col_series.apply(lambda x: df[x].dtype)
        ],
        axis=1).set_axis(['Column_Name', 'Num_Unique', 'Type'],
                         axis=1).sort_values(by='Num_Unique',
                                             ignore_index=True)
    return ucounts_df


if __name__ == '__main__':
    display(unique_counts(churn_df))

,Column_Name,Num_Unique,Type
0,Gender,2,object
1,HasCrCard,2,int64
2,IsActiveMember,2,int64
3,Exited,2,int64
4,Geography,3,object
5,NumOfProducts,4,int64
6,Tenure,11,int64
7,Age,70,int64
8,CreditScore,460,int64
9,Balance,6382,float64


In [194]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"


if __name__ == '__main__':
    churn_df['CreditScore'] = churn_df['CreditScore'].apply(
        get_credit_score_cat)

    cat_list = ['CreditScore', 'Geography', 'Gender', 'NumOfProducts']
    bool_list = ['HasCrCard', 'IsActiveMember', 'Exited']
    int_list = ['Tenure', 'Age']
    float_list = ['Balance', 'EstimatedSalary']    
    for col in churn_df.columns:
        if col in cat_list:
            churn_df[col] = churn_df[col].astype('category')
        elif col in bool_list:
            churn_df[col] = churn_df[col].apply(lambda x: x == 1)
        elif col in int_list:
            churn_df[col] = churn_df[col].astype('uint8')
        elif col in float_list:
            churn_df[col] = churn_df[col].astype('float32')            

    profit = 100 - (churn_df.memory_usage().sum() /
                    churn_data.memory_usage().sum() * 100).round(decimals=2)
    print(f'The memory usage profit after data transformation: {profit}%\n')

    display(churn_df.info())
    # churn_df.to_csv('data/churn_res.csv')  # already done


The memory usage profit after data transformation: 84.75%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      10000 non-null  category
 1   Geography        10000 non-null  category
 2   Gender           10000 non-null  category
 3   Age              10000 non-null  uint8   
 4   Tenure           10000 non-null  uint8   
 5   Balance          10000 non-null  float32 
 6   NumOfProducts    10000 non-null  category
 7   HasCrCard        10000 non-null  bool    
 8   IsActiveMember   10000 non-null  bool    
 9   EstimatedSalary  10000 non-null  float32 
 10  Exited           10000 non-null  bool    
dtypes: bool(3), category(4), float32(2), uint8(2)
memory usage: 166.8 KB


None

<div style="text-align:center"><span style="color:blue">В результате преобразований, помимо приведения данных к наиболее уместным для последующего анализа типам, достигнуто облегчение фрейма на 84.75% до 166.8 KB.</span></div>

## <div style="text-align:center"><span style="color:red">Исследование зависимостей с иллюстрациями.</span></div>

<span style="color:red">1). Соотношение ушедших и лояльных клиентов</span>

In [195]:
fig = go.Figure(data=[
    go.Pie(labels=['Лояльные', 'Ушедшие'],
           values=churn_df['Exited'].value_counts().values,
           pull=[0.01, 0.08],
           hole=.4)
])
fig.update_traces(marker=dict(colors=['forestgreen', 'red'],
                              line=dict(color='grey', width=1.5)))
fig.update_layout(width=700,
                  height=600,
                  title_text='Соотношение ушедших и лояльных клиентов',
                  legend=dict(yanchor="middle", y=0.5, xanchor="center",
                              x=0.5),
                  legend_traceorder='reversed')
fig.show()
# fig.write_html("plotly/fig_01.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_01.png?raw=true alt='fig_01.png' style='width:700px'>

- <span style="color:blue">Покинувших: 2037 клиентов из 10000 (20.4%)</span>
- <span style="color:blue">Лояльных: 7963 клиентов из 10000 (79.6%)</span>
- <span style="color:blue">Если долго удерживать взгляд в центре данной диаграммы, может показаться, что банк покидает каждый пятый клиент</span>

<span style="color:red">2). Распределение баланса пользователей с наличием на счету более 2500 условных единиц (или просто УЕ)</span>

In [196]:
churn_here = churn_df['Balance'][churn_df['Balance'] > 2500]
bal_mean = churn_here.mean()
a, b = 'Правосторонняя', 'Левосторонняя'
asym = churn_here.skew()

fig = px.histogram(
    churn_here,
    x='Balance',
    marginal='box',
    labels={'Balance': 'Баланс'},
    title='Распределение баланса клиентов, '
    'у которых на счету больше 2500 условных единиц',
    width=1200,
    height=600,
)
fig.update_layout(yaxis_title='Количество клиентов')
fig.add_annotation(text=f'Асимметрия: {asym}, {a if asym > 0 else b}',
                   font={
                       'color': '#636efa',
                       'size': 15
                   },
                   xref="x domain",
                   yref="y domain",
                   x=0.01,
                   y=0.99,
                   showarrow=False)
fig.add_vline(x=bal_mean,
              line_width=2,
              line_dash="solid",
              line_color="crimson",
              annotation_text=f'среднее: {bal_mean}',
              annotation_position="bottom right",
              annotation_font_size=15,
              annotation_font_color="crimson")
fig.show()
# fig.write_html("plotly/fig_02.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_02.png?raw=true alt='fig_02.png' style='width:1200px'>

- <span style="color:blue">Распределение нормальное с незначительной асимметрией вправо и некоторым количеством выбросов</span>
- <span style="color:blue">Значение медианы совпадает со средним, если отсечь вклады с незначительным количеством УЕ на счету</span>
- <span style="color:blue">Большая часть значимых клиентов располагает суммами от 40 до 200 тысяч УЕ (максимум - 250)</span>
- <span style="color:blue">И ни одного миллионера (не то, что в некоторых других царствах-государствах)!</span>

<span style="color:red">3). Распределение баланса клиента в разрезе признака оттока (с чем это может быть связано, что может не устраивать ушедших клиентов в банке)</span>

In [197]:
bal_mean = churn_df['Balance'].mean()

fig = px.box(
    churn_df,
    x='Balance',
    color='Exited',
    labels={
        'Balance': 'Баланс',
        'Exited': 'Признак оттока'
    },
    title='Распределение баланса клиента в разрезе признака оттока',
    width=1200,
    height=400,
)
fig.update_layout(xaxis_title='Баланс',
                  yaxis_title='Признак оттока',
                  legend_traceorder='reversed')
fig.add_vline(x=bal_mean,
              line_width=2,
              line_dash='solid',
              line_color='crimson',
              annotation_text=f'среднее: {bal_mean}',
              annotation_position="bottom right",
              annotation_font_size=15,
              annotation_font_color="crimson")
fig.show()
# fig.write_html("plotly/fig_03.html") # done (for github)

fig = px.scatter(
    churn_df,
    y='Balance',
    # y='Exited',
    color='Exited',
    opacity=0.7,
    labels={
        'Balance': 'Баланс',
        'index': 'Клиент (индекс в таблице)',
        'Exited': 'Признак оттока'
    },
    title=
    'Гистограмма рассеяния баланса клиента в разрезе признака оттока',
    width=1200,
    height=600)
fig.update_layout(legend_traceorder='reversed')
fig.show()
# fig.write_html("plotly/fig_031.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_03.png?raw=true alt='fig_03.png' style='width:1200px'>

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_031.png?raw=true alt='fig_031.png' style='width:1200px'>

- <span style="color:blue">Лояльные: медиана - 92000 УЕ, большинство - 0-126000, границы - 0-222000</span>
- <span style="color:blue">Покинувшие: медиана - 109000 УЕ, большинство - 38000-131000, границы - 0-260000</span>
- <span style="color:blue">По всему диапазону среднее ниже медианы за счёт значительного количества незначительных по суммам вкладов</span>
- <span style="color:blue">На гистограмме огромное количество клиентов с пустыми счетами сливается в одну сплошную линию</span>
- <span style="color:blue">В целом складывается устойчивое впечатление, что нелояльные клиенты как-то побогаче - даже медиана по вкладам у них больше 100000, а деньги-то это весьма немалые!</span>

<span style="color:red">4). Распределение возраста в разрезе признака оттока (В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?)</span>

In [198]:
fig = px.box(
    churn_df,
    x='Age',
    color='Exited',
    labels={
        'Age': 'Возраст',
        'Exited': 'Признак оттока'
    },
    title='Распределение возраста клиента в разрезе признака оттока',
    width=1200,
    height=400,
)
fig.update_layout(yaxis_title='Признак оттока', legend_traceorder='reversed')
fig.show()
# fig.write_html("plotly/fig_04.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_04.png?raw=true alt='fig_04.png' style='width:1200px'>

- <span style="color:blue">Лояльные: медиана - 36 лет, большинство - 31-40, границы - 18-56</span>
- <span style="color:blue">Покинувшие: медиана - 45 лет, большинство - 38-51, границы - 19-70</span>
- <span style="color:blue">Потенциальных выбросов больше в старшей возрастной группе</span>
- <span style="color:blue">Отток также больше среди людей постарше, на них и надо пробовать обращать внимание</span>
- <span style="color:blue">Задача не из лёгких - голодным неопытным студентам и молодым специалистам проще продать продукт, чем умудрённым опытом владельцам яхт, пароходов и собственных самогонных аппаратов</span>
- <span style="color:blue">Кроме того, некоторые представители старшего поколения с трудом перемещаются, да и просто не горят желанием ходить в какой-то непонятный банк, поэтому надо внедрять привлекательные и развлекательные дистанционные сервисы</span>

<span style="color:red">5). Взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты (какова взаимосвязь между признаками)</span>

In [199]:
# using here the source dataframe, untouched by data transformation,
# since the 'CreditScore' had been tranformed to the 'category' type:
churn_here = churn_data[['CreditScore', 'EstimatedSalary', 'Exited']].copy()
churn_here['Exited'] = churn_here['Exited'].apply(lambda x: x == 1)
churn_here_box = churn_df.groupby(['CreditScore', 'Exited'],
                                  as_index=False)['EstimatedSalary'].mean()

fig = px.scatter(
    churn_here,
    x='CreditScore',
    y='EstimatedSalary',
    color='Exited',
    opacity=0.7,
    labels={
        'CreditScore': 'Кредитный рейтинг',
        'EstimatedSalary': 'Предполагаемая зарплата',
        'Exited': 'Признак оттока'
    },
    title=
    'Взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты',
    width=1200,
    height=600)
fig.show()
# fig.write_html("plotly/fig_05.html") # done (for github)

fig = px.bar(
    churn_here_box,
    x='CreditScore',
    y='EstimatedSalary',
    color='Exited',
    barmode='group',
    labels={
        'CreditScore': 'Кредитный рейтинг',
        'EstimatedSalary': 'Среднее по зарплате',
        'Exited': 'Признак оттока',
    },
    title='Среднее предполагаемой ЗП по категоряим '
    'кредитного рейтинга в разрезе признака оттока',
    width=1000,
    height=400,
)
fig.show()
# fig.write_html("plotly/fig_051.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_05.png?raw=true alt='fig_05.png' style='width:1200px'>

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_051.png?raw=true alt='fig_051.png' style='width:1000px'>

- <span style="color:blue">Ну вот уж тут-то даже при задействованнии опции прозрачности библиотеки plotly ничего толком не разглядеть</span>
- <span style="color:blue">Имеется неперекрываемая зона ярко выраженной нелояльности среди клиентов с крайне низким кредитным рейтингом</span>
- <span style="color:blue">В остальном значения признака оттока распределены более-менее равномерно по всему диапазону без особенностей</span>
- <span style="color:blue">Максимум средней предполагаемой ЗП (109500 УЕ) находится среди нелояльных клиентов в самой неблагонадёжной категории кредитного рейтинга</span>
- <span style="color:blue">На втором месте по средней ЗП (103700 УЕ) расположилились самые благонадёжные лояльные клиенты, а минимум ЗП (94800) предполагается для нелояльных клиентов в той же самой, высшей категории кредитного рейтинга</span>
- <span style="color:blue">Разброс по значениям незначителен, явных закономерностей по распределению не заметно</span>

<span style="color:red">6). Кто чаще уходит, мужчины или женщины?</span>

In [200]:
churn_here = churn_df.groupby(['Gender', 'CreditScore'],
                              as_index=False)['Exited'].mean().round(2)
churn_here.replace({'Gender': {
    'Female': 'Женщины',
    'Male': 'Мужчины'
}},
                   inplace=True)

fig = px.bar(
    churn_here,
    x='CreditScore',
    y=churn_here['Exited'].mul(100),
    color='Gender',
    # category_orders={'Gender': ['Женщины', 'Мужчины']},
    barmode='group',
    hover_name='Gender',
    hover_data={
        'Exited': False,
        'Gender': False
    },
    labels={
        'Gender': 'Признак пола',
        'CreditScore': 'Кредитный рейтинг',
        'y': 'Процент оттока'
    },
    title='Процент оттока в группах по половому признаку и кредитному рейтингу',
    width=1000,
    height=400,
)
fig.show()
# fig.write_html("plotly/fig_06.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_06.png?raw=true alt='fig_06.png' style='width:1000px'>

- <span style="color:blue">Процент оттока среди женщин выше, чем среди мужчин и составляет примерно 25% против 16%</span>
- <span style="color:blue">Даже кредитный рейтинг на этом практически не сказывается</span>
- <span style="color:blue">А я всегда говорил, что вся надежда на прекрасный пол!</span>
- <span style="color:blue">И здесь повторю: Берегите женщин, они вам пригодятся!</span>

<span style="color:red">7). Как отток клиентов зависит от числа приобретённых у банка услуг?</span>

In [201]:
churn_df['index_col'] = churn_df.index
churn_here = churn_df.groupby(['NumOfProducts', 'Exited'],
                              as_index=False)['index_col'].count()

fig = px.bar(
    churn_here,
    x='NumOfProducts',
    y='index_col',
    color='Exited',
    barmode='group',
    labels={
        'NumOfProducts': 'Количество услуг',
        'Exited': 'Признак оттока',
        'index_col': 'Количество клиентов'
    },
    title='Зависимость оттока клиентов от количества приобретённых услуг',
    width=1000,
    height=400,
)
fig.show()
# fig.write_html("plotly/fig_07.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_07.png?raw=true alt='fig_07.png' style='width:1000px'>

- <span style="color:blue">Подавляющее большинство клиентов приобретает от одной до двух услуг</span>
- <span style="color:blue">1 услуга: лояльных клиентов - 3675, покинувших - 1409</span>
- <span style="color:blue">2 услуги: лояльных - 4242, покинувших - 348</span>
- <span style="color:blue">3 услуги: лояльных - 46, покинувших - 220</span>
- <span style="color:blue">4 услуги: лояльных - (sic!) ни одного, покинувших - 60</span>
- <span style="color:blue">Какие-то неизобретательные тут банкиры: не осталоось никого, кто бы использовал весь спектр услуг банка, да и было таких клиентов ничтожно мало</span>
- <span style="color:blue">Кое-кто из пользователей сразу трёх продуктов пока ещё уцелел, но покинувших больше в разы, при этом и тех, и других в сумме не более 3% от общего числа клиентов</span>
- <span style="color:blue">Значительная доля оттока наблюдается у тех, кто приобрёл всего один продукт - ничего странного: в этой группе больше всего временщиков и проходимцев, которые сами не знают, чего им надо</span>
- <span style="color:blue">А вот среди пользователей двух услуг отток составляет менее 10%, а значит у банка имеется опыт предложения действительно востребованных услуг</span>
- <span style="color:blue">Напрашивается ревизия банковских продуктов</span>

<span style="color:red">8). Как влияет наличие статуса активного клиента на отток клиентов? Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?</span>

In [202]:
churn_here = churn_df.groupby(['IsActiveMember', 'Exited'],
                              as_index=False)['index_col'].count()

fig = px.bar(
    churn_here,
    x='IsActiveMember',
    y='index_col',
    color='Exited',
    barmode='group',
    labels={
        'IsActiveMember': 'Признак активности',
        'Exited': 'Признак оттока',
        'index_col': 'Количество клиентов'
    },
    title='Взаимосвязь признаков активности и оттока',
    width=800,
    height=400,
)
fig.show()
# fig.write_html("plotly/fig_08.html") # done (for github)
churn_df.drop('index_col', axis=1, inplace=True)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_08.png?raw=true alt='fig_08.png' style='width:800px'>

- <span style="color:blue">Среди активных отток 775 клиентов против 4416 лояльных</span>
- <span style="color:blue">Среди неактивных отток 1302 клиентов против 3547</span>
- <span style="color:blue">Половина клиентов банка не проявляет активности</span>
- <span style="color:blue">Помимо гениальных экспертных соображений и рекомендаций, изложенных в предыдущих пунктах, можно посоветовать банку предпринять шаги для оживления конъюнктуры</span>
- <span style="color:blue">Для пробуждения активности надо вкладываться в такие процессы, как геймификация, разработка программ лояльности или расширение диапазона продуктов</span>
- <span style="color:blue">А то, видишь ли, набрали себе клиентов и сидят в своём офисе, бумажки перекладывают, посмотрите-ка на них!</span>

<span style="color:red">9). В какой стране доля ушедших клиентов больше? Предположите, с чем это может быть связано.</span>

In [203]:
churn_here = churn_df.groupby('Geography')['Exited'].agg(['mean', 'count'
                                                          ]).reset_index()
churn_here['mean'] = churn_here['mean'].mul(100).round(2)
churn_here['count'] = churn_here['count'].div(
    churn_here['count'].sum()).mul(100).round(2)

fig = px.choropleth(
    churn_here,
    locations='Geography',
    locationmode='country names',
    color='mean',
    color_continuous_scale='ylorrd',
    labels={
        'count': 'Клиентов в проц. от всех',
        'mean': 'Процент оттока'
    },
    hover_name='Geography',
    hover_data={
        'Geography': False,
        'count': True,        
        'mean': True
    },
    range_color=[0, 33],
    title='Распределение признака оттока по странам в процентах',
    width=1000,
    height=600)
fig.show()
# fig.write_html("plotly/fig_09.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_09.png?raw=true alt='fig_09.png' style='width:1000px'>

- <span style="color:blue">Доля ушедших в одной немецкоговорящей стране в два раза больше, чем в в каждой из двух франко- и испаноговорящих других - 32% против 16% и 17% соответственно</span>
- <span style="color:blue">Всего клиентов в этой стране 25.09%, а в двух других — 50.14% и 24.77% соответственно (разглядеть можно при наведении в нативной среде Jupyter, без него не видно)</span>
- <span style="color:blue">По всей видимости, близость Средиземного моря и Атлантики оказывает на клиентов успокаивающее, а местные виноградники - длительное пьянящее воздействие</span>
- <span style="color:blue">В более рассчётливом, суровом немецком социуме за это время понаоткрывалось столько оборотистых банков, что плоду с подстриженного фруктового дерева упасть некуда!</span>
- <span style="color:blue">Как говорится, чтобы оставаться на месте, нынче приходится быстро бежать, а лучше сразу приобрести у наших партнёров надёжный немецкий велосипед (ой, это, кажется, не сюда, сорри)!</span>

<span style="color:red">10). Тепловая карта по значениям процента оттока для пересечений признаков кредитного рейтинга и клиентского стажа</span>

In [204]:
churn_here = churn_df.pivot_table(values='Exited',
                                  index='CreditScore',
                                  columns='Tenure',
                                  aggfunc='mean',
                                  fill_value=0).round(decimals=2).mul(100)

fig = px.imshow(
    churn_here,
    text_auto=True,
    labels=dict(x="Клиентский стаж (лет)",
                y="Кредитный рейтинг",
                color="Процент оттока"),
    color_continuous_scale='ylorrd',
    title='Тепловая карта по значениям процента оттока для пересечений '
    'признаков кредитного рейтинга и клиентского стажа',
    width=1200,
    height=400)
fig.show()
# fig.write_html("plotly/fig_10.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/pу13_task/plotly/fig_10.png?raw=true alt='fig_10.png' style='width:1200px'>

- <span style="color:blue">Несмотря на некоторый разброс по значениям, чаще других уходят клиенты с низким кредитным рейтингом</span>
- <span style="color:blue">Наибольшее значение по признаку оттока наблюдается для предельных случаев клиентского стажа</span>
- <span style="color:blue">Среди них больше всего тех, кто только начал пользоваться услугами банка, но не стал продолжать</span>
- <span style="color:blue">На втором месте те, у кого стаж наибольший - целых 10 лет (с ума сойти)!</span>
- <span style="color:blue">Ну, в общем, такой себе банк - заточен на один-два продукта, не знает хорошего обращения с женщинами, а немцы от него уже просто шарахаются!</span>
- <span style="color:blue">Высокие показатели оттока и не самая высокая активность даже среди лояльных пользователей наводит на мысли о необходимости кардинального пересмотра клиентской политики</span>